# Constant Variable

In [33]:
import torch

random_state = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


# Data Preparation

In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

file_path = ""

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [50]:
import pandas as pd

print(file_path)

data = pd.read_csv(file_path)

df = pd.DataFrame(data)
df.head()

/kaggle/input/breast-cancer-dataset/data.csv


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [55]:
X = data.iloc[:, 2:32].values

from sklearn.preprocessing import LabelEncoder
# y = data.iloc[:, 1:2].values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data.iloc[:, 1].values)

print(X[:10])
print(y[:10])

[[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
  1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
  6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
  1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
  4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 1.326e+03 8.474e-02 7.864e-02 8.690e-02
  7.017e-02 1.812e-01 5.667e-02 5.435e-01 7.339e-01 3.398e+00 7.408e+01
  5.225e-03 1.308e-02 1.860e-02 1.340e-02 1.389e-02 3.532e-03 2.499e+01
  2.341e+01 1.588e+02 1.956e+03 1.238e-01 1.866e-01 2.416e-01 1.860e-01
  2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 1.203e+03 1.096e-01 1.599e-01 1.974e-01
  1.279e-01 2.069e-01 5.999e-02 7.456e-01 7.869e-01 4.585e+00 9.403e+01
  6.150e-03 4.006e-02 3.832e-02 2.058e-02 2.250e-02 4.571e-03 2.357e+01
  2.553e+01 1.525e+02 1.709e+03 1.444e-01 4.245e-01 4.504e-01 2.430e-01
  3.613e-01 8.758e-02]
 [1.142e+01 2.038e+01 7.758e+01 3.861e+02 1.425e-01 2.839e-01 2.414

# Dataset Splitting

In [56]:
from sklearn.model_selection import train_test_split

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

455
114
455
114


In [57]:
X_train.shape, y_train.shape

(torch.Size([455, 30]), torch.Size([455]))

# Build Neural Network Binary Classification Model

In [101]:
import torch
from torch import nn

class BinaryClassificationModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=30, out_features=64)
        self.layer_2 = nn.Linear(in_features=64, out_features=16)
        self.layer_3 = nn.Linear(in_features=16, out_features=1)
        self.relu = nn.ReLU()
        
    def forward(self, X):
        z = self.layer_1(X)
        z = self.relu(z)
        z = self.layer_2(z)
        z = self.relu(z)
        z = self.layer_3(z)
        return z

model_0 = BinaryClassificationModelV0().to(device)
model_0

BinaryClassificationModelV0(
  (layer_1): Linear(in_features=30, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=16, bias=True)
  (layer_3): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

# Setup Loss function & Optimizer

In [102]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(params=model_0.parameters(),
                           lr=0.01)

In [78]:
!pip -q install torchmetrics
from torchmetrics import Accuracy

acc_fn = Accuracy(task="multiclass", num_classes=2).to(device)
acc_fn

MulticlassAccuracy()

# Training & Testing loop

In [103]:
torch.manual_seed(random_state)
torch.cuda.manual_seed(random_state)

epochs = 100

# Place data into target device
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

for epoch in range(epochs):
    model_0.train()
    
    y_logits = model_0(X_train).squeeze()
    y_pred_prob = torch.sigmoid(y_logits)
    y_pred = torch.round(y_pred_prob)
    
    loss = loss_fn(y_logits, y_train)
    acc = acc_fn(y_pred, y_train)
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    model_0.eval()
    with torch.inference_mode():
        test_logits = model_0(X_test).squeeze()
        test_pred_prob = torch.sigmoid(test_logits)
        test_pred = torch.round(test_pred_prob)
        
        test_loss = loss_fn(test_logits, y_test)
        test_acc = acc_fn(test_pred, y_test)
    
    if epoch % 10 == 0 or epoch == epochs - 1:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {acc:.2f}% | Test Loss: {test_loss:.5f} | Loss Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 7.45986 | Acc: 0.37% | Test Loss: 3.96109 | Loss Acc: 0.62%
Epoch: 10 | Loss: 0.88204 | Acc: 0.40% | Test Loss: 0.80815 | Loss Acc: 0.41%
Epoch: 20 | Loss: 0.50609 | Acc: 0.73% | Test Loss: 0.39339 | Loss Acc: 0.90%
Epoch: 30 | Loss: 0.36219 | Acc: 0.88% | Test Loss: 0.27486 | Loss Acc: 0.93%
Epoch: 40 | Loss: 0.29567 | Acc: 0.91% | Test Loss: 0.21121 | Loss Acc: 0.94%
Epoch: 50 | Loss: 0.25869 | Acc: 0.91% | Test Loss: 0.17382 | Loss Acc: 0.93%
Epoch: 60 | Loss: 0.24644 | Acc: 0.91% | Test Loss: 0.15565 | Loss Acc: 0.94%
Epoch: 70 | Loss: 0.23928 | Acc: 0.91% | Test Loss: 0.14237 | Loss Acc: 0.95%
Epoch: 80 | Loss: 0.23450 | Acc: 0.91% | Test Loss: 0.13525 | Loss Acc: 0.95%
Epoch: 90 | Loss: 0.22967 | Acc: 0.91% | Test Loss: 0.13121 | Loss Acc: 0.96%
Epoch: 99 | Loss: 0.22504 | Acc: 0.92% | Test Loss: 0.12902 | Loss Acc: 0.95%
